### Замыкания (часть первая)

Замыканием называется такая ситуация когда вложенная функция пользуется переменными не объявленными в ее теле  
Замыкание — это особый вид функции. Она определена в теле другой функции и создаётся каждый раз во время её выполнения.

In [6]:
def main_func():

    def inner_func():  # Вложенная функция
        print('hello world')
    # Возвращаем вложенную функцию(важно что не значение которок возвращает а именно функцию)
    return inner_func


a = main_func()  # После этого ссылается уже на функцию и по сути сама переменная становится функцией
a()
a


hello world


<function __main__.main_func.<locals>.inner_func()>

In [14]:
def main_func(value):
    name = value

    def inner_func():
        # Тут функция будет находить переменную в родительской функции
        print('hello world', name)
    return inner_func


b = main_func('Serega')
b()  # Получается что тут переменная уже связан с именем Серега
v = main_func('Vasya')  # Получается что создалось две области видиммости
v()
b


hello world Serega
hello world Vasya


<function __main__.main_func.<locals>.inner_func()>

In [25]:
def main_func(name):  # Стоит отметить что тут убрали присвоение name в теле функции
    def inner_func():
        # Тут функция будет находить переменную в родительской функции
        print('hello world', name)
    return inner_func


g = main_func('Leha')
g()
g


hello world Leha


<function __main__.main_func.<locals>.inner_func()>

In [27]:
def adder(value):
    def inner(a):
        return value+a
    return inner

a2 = adder(2)
a2

<function __main__.adder.<locals>.inner(a)>

In [28]:
a2(5)

7

In [30]:
def counter():
    count = 0

    def inner():
        nonlocal count  # nonlocal указывает что ссылаемся не на локальную перенную
        count += 1  # Также необходима чтоб переменную можно изменяь в другой области видимости
        return count

    return inner

c = counter()
print(c())  # получается каждый раз вызывая одну функцию, мы меняем count
print(c())
print(c())
print(c())


1
2
3
4


### Замыкание (часть два)

In [8]:
def average_numbers():
    numbers = []

    def inner(number):
        numbers.append(number)
        print(numbers)
        return sum(numbers) / len(numbers)
    return inner

r1 = average_numbers()
print(r1(5))
print(r1(3))
print(r1(8))


[5]
5.0
[5, 3]
4.0
[5, 3, 8]
5.333333333333333


In [38]:
r1(10)
d2 = average_numbers()
d2(100)

[5, 3, 10]
[100]


100.0

In [43]:
def average_numbers_v2():
    summa = 0
    count = 0

    def inner(number):
        nonlocal summa
        nonlocal count
        summa += number
        count += 1
        print(f'summa = {summa} | count = {count} | mean = {summa/count}')
        return summa/count
    return inner


d1 = average_numbers_v2()
d1(10)
d1(5)
d1(22)


summa = 10 | count = 1 | mean = 10.0
summa = 15 | count = 2 | mean = 7.5
summa = 37 | count = 3 | mean = 12.333333333333334


12.333333333333334

In [24]:
from datetime import datetime


def timer():
    start = datetime.now()

    def counting():
        return (datetime.now() - start)

    return counting


t1 = timer()
c = 0
for i in range(5):
    c += i
print(t1())


0:00:00


In [25]:
print(t1())

0:00:00.730642


In [27]:
from time import perf_counter

In [28]:
def timer_v2():
    start = perf_counter()

    def inner():
        return perf_counter() - start
    return inner


t2 = timer_v2()


In [31]:
t2()

3.2743083000000297

In [34]:
def timer_v2():
    now = perf_counter()

    def inner():
        nonlocal now
        time = perf_counter() - now
        now = perf_counter()
        return time
    return inner


t2 = timer_v2()
t2()

2.9900000072302646e-05

In [38]:
t2()

1.9999760000000606

In [74]:
t2()
c = 1
for i in range(1, 100001):
    c *= i
# print(c)
t2()

3.250231600002735

In [81]:
def add(a, b):
    return a+b

def mult(a, b, c):
    return a*b*c

def counter(func):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'функция {func.__name__} вызывалась {count} раз {func(*args, **kwargs)}')
        return func(*args, **kwargs)
    return inner


c1 = counter(add)
c1(10, 20)
c1(10, 30)
d1 = counter(mult)
d1(10, 20, 30)
d1(3, 5, 7)


функция add вызывалась 1 раз 30
функция add вызывалась 2 раз 40
функция mult вызывалась 1 раз 6000
функция mult вызывалась 2 раз 105


105

In [75]:
abs.__name__  # Переменная что получить имя функции

'abs'

### Декораторы

Определение декоратора : это функция которое принимает функцию и возращает функцию  
Декораторы нужны чтоб функции добавилось новое поведение или новый функционал

In [97]:
def decorator(func):

    # при использование декоратора, завести за правило использовать *args & **kwargs
    def inner(*args, **kwargs):
        print('start decorator: ...')
        func(*args, **kwargs)
        print('finish decorator: ...')
    return inner


def say():
    print('hello world')


d = decorator(say)
print(d)
d()
print()
print('А тут мы подменяем уже функцию say но новую!!!:')
say = decorator(say)
say()


<function decorator.<locals>.inner at 0x000002A290D73E50>
start decorator: ...
hello world
finish decorator: ...

А тут мы подменяем уже функцию say но новую!!!:
start decorator: ...
hello world
finish decorator: ...


In [89]:
def decorator(func):
    
    def inner():
        print('start decorator: ...')
        func()
        print('finish decorator: ...')
    return inner

def say():
    print('hello world')
    
def buy():
    print('buy world')
    
new_buy = decorator(buy)
new_buy()

start decorator: ...
buy world
finish decorator: ...


In [95]:
def decorator(func):
    
    def inner(*args, **kwargs):
        print('start decorator: ...')
        func(*args, **kwargs)
        print('finish decorator: ...')
    return inner

def say(name):
    print('hello', name)
    
new_say = decorator(say)
new_say('Leha')

start decorator: ...
hello Leha
finish decorator: ...


In [4]:
def header(func):
    
    def inner(*args, **kwargs):
        print('<h1>')
        func(*args, **kwargs)
        print('</h1>')
    return inner


def table(func):
    
    def inner(*args, **kwargs):
        print('<table>')
        func(*args, **kwargs)
        print('</table>')
    return inner

def print_text(text):
    print(text)

print_text = table(header(print_text))
print_text('Здесь какой то текст, который надо обернуть')

<table>
<h1>
Здесь какой то текст, который надо обернуть
</h1>
</table>


Но правильнее это записать:

In [ ]:
@table
@header
def print_text(text):
    print(text)
    
print_text('Новый текст')


<table>
<h1>
Новый текст
</h1>
</table>


Возникает проблема потери оригинальной функции и ее документации  
Первый вариант решения проблемы:

In [ ]:
def header(func):
    
    def inner(*args, **kwargs):
        print('<h1>')
        func(*args, **kwargs)
        print('</h1>')
        
    inner.__name__ = func.__name__ # подменяем, чтоб сохранить имя и документацию
    inner.__func__ = func.__doc__
    return inner

In [41]:
def matreshka(n):
    if n == 1:
        print('Матрешка')
    else:
        print(f'Верх матрёшки {n}')
        matreshka(n-1)
        print(f'Низ матрёшки {n}')
        
matreshka(5)

Верх матрёшки 5
Верх матрёшки 4
Верх матрёшки 3
Верх матрёшки 2
Матрешка
Низ матрёшки 2
Низ матрёшки 3
Низ матрёшки 4
Низ матрёшки 5


In [48]:
class Decorat():
    def __init__(self):
        self.name = 'This is name'
        
    def decorat_print(value):
        def decor_inside(func):
            def inner(*args, **kwargs):
                print(args[0].name)
                print('just text')
                print("it's value:", value)
                func(*args, **kwargs)

            return inner
        return decor_inside
    
    @decorat_print(5)
    def method_first_class(self, text):
        print(text)
        
a = Decorat()
a.method_first_class('Это функция')

This is name
just text
it's value: 5
Это функция


In [50]:
class new_class(Decorat):
    def __init__(self, name):
        self.name = name
        
b = new_class('B!!!')
b.method_first_class('Декорированная функция')

B!!!
just text
it's value: 5
Декорированная функция


In [ ]:
from functools import wraps # второй вариант решения проблемы. использования декоратора wraps

def header(func):
    
    @wraps(func) # Декорируем декорируемую функцию)
    def inner(*args, **kwargs):
        print('<h1>')
        func(*args, **kwargs)
        print('</h1>')

    return inner

поставил на паузу, пошел смотреть про `args` и `kwargs`  
[ссылка на видео 6.47](https://www.youtube.com/watch?v=Va-ovLxHmus)